# Import statements

In [42]:
# Cell 1: Import Libraries
import geopandas as gpd
import pandas as pd
import glob
import os
import json
import plotly.express as px
import dash
from dash import dcc, html, Input, Output, Dash

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import pandas as pd
# Set SHAPE_RESTORE_SHX environment variable
os.environ["SHAPE_RESTORE_SHX"] = "YES"

In [43]:
# Read the JSON file
def read_json(filename):
    with open(filename, 'r') as file:
        gemmingen_data = json.load(file)
    return gemmingen_data

In [44]:
def open_gdf_file(file_path):
    gdf = gpd.read_file(file_path)
    return gdf

In [45]:
#util files
def convert_units(value, from_unit, to_unit):
    """
    Converts the value between supported units.
    For simplicity, this function supports conversion between:
    - 'ml1ha-1' and 'l1ha-1'
    """
    conversion_factors = {
        ("ml1ha-1", "l1ha-1"): 0.001,  # 1 ml = 0.001 l
        ("l1ha-1", "ml1ha-1"): 1000,   # 1 l = 1000 ml
        ("g1ha-1", "kg1ha-1"): 0.001,
        ("kg1ha-1", "g1ha-1"): 1000,
        ("g1ha-1", "l1ha-1"): 0.001,
        ("ml1ha-1", "kg1ha-1"): 0.001
    }
    
    if from_unit == to_unit:
        return value  # No conversion needed
    key = (from_unit, to_unit)
    if key in conversion_factors:
        return value * conversion_factors[key]
    raise ValueError(f"Unsupported unit conversion from {from_unit} to {to_unit}")

def process_product_data(product_data):
    if not product_data["TankMix"]:
        return "TankMix is False; no processing required."
    
    # Step 2: Get the rate value and unit
    rate_value = product_data["Rate"]["Value"]
    rate_unit = product_data["Rate"]["Unit"]
    
    # Step 3: Go into the components and find carriers == False
    components = product_data["Components"]
    carrier_sum = 0
    for component in components:
        if not component["Carrier"]:  # Check if carrier is False
            component_rate = component["Rate"]["Value"]
            component_unit = component["Rate"]["Unit"]
            
            # Step 4: Convert unit if different
            component_rate_converted = convert_units(component_rate, component_unit, rate_unit)
            carrier_sum += component_rate_converted
    
    # Step 5: Divide the summation by rate value
    result = carrier_sum / rate_value
    return result

In [5]:
# # Cell 2: Define Root Folder and Initialize List
# root_folder = "data/Erntejahr 2024 (Weizen, komplett)"

# calculate_emissions_json = read_json("gemmingen.json")
# gemmingen_area = fertilizer_json["area"]

# # Initialize an empty list to store shapefile paths
# shapefile_list = []

# # Traverse the directory structure
# for subdir, _, _ in os.walk(root_folder):
#     # Look for .shp files in the 'doc' folder of each subdirectory
#     shapefiles = glob.glob(os.path.join(subdir, "doc", "*.shp"))
#     shapefile_list.extend(shapefiles)


# # Cell 3: Read Shapefiles and Combine into a Single GeoDataFrame
# gdfs = []
# for shapefile_path in shapefile_list:
#     gdf = open_gdf_file(shapefile_path)
#     # Append the GeoDataFrame to a list
#     gdfs.append(gdf)

# # Concatenate all GeoDataFrames into a single GeoDataFrame
# combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# # Fill missing values with NA
# combined_gdf = combined_gdf.fillna(0)

# # Cell 4: Display Output and Columns
# print("All shapefiles combined successfully!")

In [46]:
root_folder = "data/Erntejahr 2024 (Weizen, komplett)"
json_file = "calculation.json"
Farm_name = "Gemmingen"

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display

# Define the root data folder
data_folder = "data"
# Get a list of all folders in the data folder
folders = [f for f in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, f))]
Farms = ["Gemmingen", "Jennewein"]
# Create a dropdown widget with the folders
dropdown = widgets.Dropdown(
    options=folders,
    description='Folders:',
    disabled=False,
)

dropdown2 = widgets.Dropdown(
    options=Farms,
    description='Farms:',
    disabled=False,
)

def on_dropdown_change_2(change):
    print(f"Selected Farm: {change.new}")
    global Farm_name
    Farm_name = change.new

# Define a callback function to read the dropdown value
def on_dropdown_change(change):
    print(f"Selected folder: {change.new}")
    global  root_folder
    root_folder = os.path.join(data_folder, change.new)

# Attach the callback function to the dropdown widget
dropdown.observe(on_dropdown_change, names='value')
dropdown2.observe(on_dropdown_change_2, names='value')
# Display the dropdown
display(dropdown2)
display(dropdown)

Dropdown(description='Farms:', options=('Gemmingen', 'Jennewein'), value='Gemmingen')

Dropdown(description='Folders:', options=('JenneweinAnwendung23', 'JenneweinErnte22', 'Erntejahr 2022 (Weizen,…

Selected Farm: Jennewein
Selected Farm: Gemmingen
Selected folder: Erntejahr 2023 (Mais, Düngung)
Selected folder: Erntejahr 2024 (Weizen, komplett)
Selected Farm: Jennewein
Selected folder: JenneweinAnwendung22
Selected folder: JenneweinErnte21
Selected folder: JenneweinAnwendung21
Selected folder: JenneweinAnwendung22
Selected folder: JenneweinErnte22
Selected folder: JenneweinAnwendung23
Selected folder: JenneweinErnte22
Selected folder: JenneweinAnwendung21
Selected folder: JenneweinErnte21
Selected folder: JenneweinAnwendung22
Selected folder: JenneweinErnte22
Selected folder: JenneweinAnwendung23
Selected folder: JenneweinErnte23
Selected folder: Jennewein Anwendung24
Selected folder: Jennewein Bestellung24
Selected folder: Jennewein Ernte24
Selected Farm: Gemmingen
Selected folder: Erntejahr 2022 (Weizen, Düngung)
Selected folder: Erntejahr 2023 (Mais, Düngung)
Selected folder: Erntejahr 2024 (Weizen, komplett)


In [165]:
# Set SHAPE_RESTORE_SHX environment variable
os.environ["SHAPE_RESTORE_SHX"] = "YES"

# Cell 2: Define Root Folder and Initialize List
# root_folder = "data/Erntejahr 2024 (Weizen, komplett)"
# json_file = "calculation.json"
# Farm_name = "Gemmingen"
fertilizer_json = read_json(json_file)
area = fertilizer_json["areas"][Farm_name]

# Initialize an empty list to store shapefile paths
shapefile_list = []
# Traverse the directory structure
for subdir, _, _ in os.walk(root_folder):
    # Look for .shp files in the 'doc' folder of each subdirectory
    shapefiles = glob.glob(os.path.join(subdir, "doc", "*.shp"))
    shapefile_list.extend(shapefiles)

# Cell 3: Read Shapefiles and Combine into a Single GeoDataFrame
gdfs = []
for i, shapefile_path in enumerate(shapefile_list):
    gdf = gpd.read_file(shapefile_path)
    # Append the GeoDataFrame to a list
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into a single GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Fill missing values with NA
combined_gdf = combined_gdf.fillna(0)

# Cell 4: Display Output and Columns
print("All shapefiles combined successfully!")

All shapefiles combined successfully!


In [166]:
# combined_gdf['Product'] = combined_gdf['Product'].apply(lambda x: x.split()[0])

# Cell 6: Calculate Emissions for Each Product
def calculate_emissions(row):
    product = row["Product"]
    appliedRate = row["AppliedRate"]
    if product in fertilizer_json:
        if fertilizer_json[product]["Zusammensetzung"]:
            if isinstance(fertilizer_json[product]["Zusammensetzung"], dict):
                composition = fertilizer_json[product]["Zusammensetzung"]["N"]
            else:
                composition = fertilizer_json[product]["Zusammensetzung"]
            emissions_factor = fertilizer_json[product]["Emissionsfaktor"]
            emissions = composition * appliedRate * emissions_factor
        else:
            emissions = appliedRate * fertilizer_json[product]["Emissionsfaktor"]
    else:
        emissions = 0
    return emissions

if "Product" in combined_gdf.columns and "AppliedRate" in combined_gdf.columns:
    print("Calculating emissions...")
    combined_gdf["Emissions"] = combined_gdf.apply(calculate_emissions, axis=1)
else:
    combined_gdf["Emissions"] = 0
# combined_gdf["Emissions"] = combined_gdf.apply(calculate_emissions, axis=1)
if "FUEL" in combined_gdf.columns:
    combined_gdf["FuelEmissions"] = combined_gdf["FUEL"] * fertilizer_json["FUEL"]["Emissionsfaktor"]
else:
    combined_gdf["FuelEmissions"] = 0
if "VRYIELDMAS" in combined_gdf.columns and "DRYMATTER" in combined_gdf.columns:
    combined_gdf["YieldEmissions"] = combined_gdf["VRYIELDMAS"] * combined_gdf["DRYMATTER"] * fertilizer_json["yield"]["Zusammensetzung"] * fertilizer_json["yield"]["Emissionsfaktor"]
else:
    combined_gdf["YieldEmissions"] = 0
# combined_gdf["YieldEmissions"] = combined_gdf["VRYIELDMAS"] *  combined_gdf["DRYMATTER"] * fertilizer_json["yield"]["Zusammensetzung"] * fertilizer_json["yield"]["Emissionsfaktor"]

Calculating emissions...


In [ ]:
# average_emissions = combined_gdf['Emissions'].mean() + combined_gdf['FuelEmissions'].mean() + combined_gdf['YieldEmissions'].mean()
# print(average_emissions * area)

1631.6038379248103


In [167]:
total_emissions = 0
total_yield_emissions = 0
total_fuel_emissions = 0
if "Product" in combined_gdf.columns:
    combined_gdf_grouped = combined_gdf.groupby("Product").agg({"Emissions": "mean"}) * area
if "FUEL" in combined_gdf.columns:
    combined_gdf_nonzero_fuel = combined_gdf[combined_gdf["FUEL"] != 0]
    combined_gdf_grouped_fuel = combined_gdf_nonzero_fuel.agg({"FuelEmissions": "mean"}) * area
if "VRYIELDMAS" in combined_gdf.columns and "DRYMATTER" in combined_gdf.columns:
    combined_gdf_nonzero_yield = combined_gdf[combined_gdf["VRYIELDMAS"] != 0]
    combined_gdf_grouped_yield = combined_gdf_nonzero_yield.agg({"YieldEmissions": "mean"}) * area
total_emissions = combined_gdf_grouped.sum()
total_fuel_emissions = combined_gdf_grouped_fuel.sum()
total_yield_emissions = combined_gdf_grouped_yield.sum()

In [168]:
total_farm_emissions = total_emissions + total_fuel_emissions + total_yield_emissions
total_farm_emissions

Emissions    3.999965e+07
dtype: float64

In [169]:
# Cell 5: Save Combined GeoDataFrame as a Shapefile
output_shapefile_path = os.path.join(root_folder, "combined_data.geojson")
combined_gdf.to_file(output_shapefile_path,driver="GeoJSON")

In [ ]:
# def gen_dash_plot(gdf, columnName, title):
#     # Convert the GeoDataFrame geometry to latitude and longitude
#     gdf["lon"] = gdf.geometry.centroid.x
#     gdf["lat"] = gdf.geometry.centroid.y

#      # Get the minimum and maximum values of the column
#     min_val = gdf[columnName].min()
#     max_val = gdf[columnName].max()

#     scatter_fig = px.scatter_mapbox(
#         gdf,
#         lat="lat",
#         lon="lon",
#         color=columnName,
#         hover_name=columnName,
#         hover_data={"lon": ":.5f", "lat": ":.5f"},  # Display coordinates with 5 decimal precision
#         title=title,
#         mapbox_style="open-street-map",
#         zoom=15,
#         height=600,
#         range_color=[min_val, max_val],
#     )
#     scatter_fig.update_traces(marker=dict(size=10, opacity=0.7))  # Customize marker size and opacity
#     return scatter_fig


In [ ]:
# # Create a Dash app
# app = Dash(__name__)

# # Sample GeoDataFrame (replace with your actual data)
# # gdf = gpd.read_file("path_to_your_shapefile.shp")
# columns = ["FUEL", "AppliedRate", "Emissions", "FuelEmissions", "YieldEmissions"]  # Non-geometry columns

# # App layout
# app.layout = html.Div([
#     html.H1("Interactive Scatter Mapbox", style={'text-align': 'center'}),

#     # Dropdown for selecting the column to color by
#     html.Div([
#         html.Label("Select Column to Visualize:"),
#         dcc.Dropdown(
#             id="color-dropdown",
#             options=[{"label": col, "value": col} for col in columns],
#             value=columns[0],  # Default value
#             clearable=False,
#         )
#     ], style={'width': '50%', 'margin': 'auto'}),

#     # Map visualization
#     dcc.Graph(id="mapbox-plot"),
# ])

# # Callback to update the scatter plot dynamically
# @app.callback(
#     Output("mapbox-plot", "figure"),
#     Input("color-dropdown", "value")
# )
# def update_mapbox(selected_column):
#     return gen_dash_plot(combined_gdf, selected_column, title=f"Scatter Mapbox - {selected_column}")

# # Run the app
# if __name__ == "__main__":
#     app.run_server(debug=True)


FertilizerAAS

Kalkammonsalpeter
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/carbonEmission/$value

Harnstoff
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTIwNV8xMjIwXzIxNDJfNzAzMw==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTIwNV8xMjIwXzIxNDJfNzAzMw==/submodel-elements/carbonEmission/$value

Ammoniumsulfatsalpeter
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMjEzNV8xMjIwXzIxNDJfMjI5OQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMjEzNV8xMjIwXzIxNDJfMjI5OQ==/submodel-elements/carbonEmission/$value

Diammonphosphat
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNzI3NV8xMjIwXzIxNDJfOTMyNQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNzI3NV8xMjIwXzIxNDJfOTMyNQ==/submodel-elements/carbonEmission/$value

Gülle
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTExMF8yMjIwXzIxNDJfODI4NQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTExMF8yMjIwXzIxNDJfODI4NQ==/submodel-elements/carbonEmission/$value


FuelAAS

dataAttributes
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzQ2M18yMjIwXzIxNDJfNTcwMA==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzQ2M18yMjIwXzIxNDJfNTcwMA==/submodel-elements/carbonEmission/$value

YieldAAS

dataAttributes
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzA3Ml8yMjIwXzIxNDJfOTgxNw==/submodel-elements/yieldMass/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzA3Ml8yMjIwXzIxNDJfOTgxNw==/submodel-elements/dryMatter/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzA3Ml8yMjIwXzIxNDJfOTgxNw==/submodel-elements/carbonEmission/$value

CropProtection

GeneralPesticides
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDMxNF8xMDMwXzIxNDJfNDA0OA==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDMxNF8xMDMwXzIxNDJfNDA0OA==/submodel-elements/carbonEmission/$value

Fungicides
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTEzNF8xMDMwXzIxNDJfNTk2NQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTEzNF8xMDMwXzIxNDJfNTk2NQ==/submodel-elements/carbonEmission/$value

Herbicides
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzMzNF8xMDMwXzIxNDJfMzQxMQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzMzNF8xMDMwXzIxNDJfMzQxMQ==/submodel-elements/carbonEmission/$value

Insecticides
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNzUzNF8xMDMwXzIxNDJfMTg2OA==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNzUzNF8xMDMwXzIxNDJfMTg2OA==/submodel-elements/carbonEmission/$value




In [ ]:
import requests
import time
# Define the URL for the PATCH request
value_url = "http://192.168.0.178:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/amount/$value"
emission_url = "http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/carbonEmission/$value"

# Define the headers, if required
headers = {
    "Content-Type": "application/json",  # Adjust based on the API's requirement
}

# Define the string to send in the body
body = "This is a string to send in the PATCH request"

# Send the PATCH request

# Read the combined_gdf product field and check if it contains the Kalkammonsalpeter and read the AppliedRate as str
if "Kalkamonsalpeter 27" in combined_gdf["Product"].values:
    applied_rate_str = combined_gdf.loc[combined_gdf["Product"] == "Kalkamonsalpeter 27", "AppliedRate"].astype(str).values
    # body = json.dumps({"value": applied_rate_str})

    for value in applied_rate_str:
        response = requests.patch(value_url, json=value, headers=headers)
        # Check the status code
        if response.status_code == 204:
            print("PATCH request successful!")
            print("Response:", response.text)
        else:
            print(f"Failed with status code {response.status_code}: {response.text}")
#     response = requests.patch(value_url, data=body, headers=headers)
#     # Check the status code
#     if response.status_code == 200:
#         print("PATCH request successful!")
#         print("Response:", response.text)
#     else:
#         print(f"Failed with status code {response.status_code}: {response.text}")


In [ ]:
import requests
import json

url = "http://192.168.0.178:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/amount/$value"
payload = "121"
response = requests.patch(url, json=payload, headers=headers)
try:
    print(f"Sent PATCH to {url} - Status Code: {response.status_code}, Response: {response.json()}")
except json.JSONDecodeError:
    print(f"Sent PATCH to {url} - Status Code: {response.status_code}, Response Text: {response.text}")

Sent PATCH to http://192.168.0.178:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/amount/$value - Status Code: 204, Response Text: 


In [ ]:
combined_gdf['Time'] = pd.to_datetime(combined_gdf['Time'])

# Add a `Year` column for grouping
combined_gdf['Year'] = combined_gdf['Time'].dt.year

# --- Step 1: Aggregate Data ---
aggregated_data = combined_gdf.groupby('Year').agg({
    'Emissions': 'sum',           # Total emissions per year
    'FuelEmissions': 'sum',       # Total fuel emissions per year
    'YieldEmissions': 'sum',      # Total yield emissions per year
    'WetMass': 'mean',            # Average wet mass
    'DRYMATTER': 'mean',          # Average dry matter
}).reset_index()

# --- Step 2: Create Charts ---
# Create a bar chart for CO2 emissions
fig1, ax1 = plt.subplots()
ax1.bar(aggregated_data['Year'], aggregated_data['Emissions'], color='skyblue')
ax1.set_title('Annual CO2 Emissions (kg CO2e)')
ax1.set_xlabel('Year')
ax1.set_ylabel('Emissions (kg CO2e)')

# Create a bar chart for fuel emissions
fig2, ax2 = plt.subplots()
ax2.bar(aggregated_data['Year'], aggregated_data['FuelEmissions'], color='orange')
ax2.set_title('Fuel Emissions (kg CO2e)')
ax2.set_xlabel('Year')
ax2.set_ylabel('Fuel Emissions (kg CO2e)')

# Create a bar chart for yield emissions
fig3, ax3 = plt.subplots()
ax3.bar(aggregated_data['Year'], aggregated_data['YieldEmissions'], color='green')
ax3.set_title('Yield Emissions (kg CO2e)')
ax3.set_xlabel('Year')
ax3.set_ylabel('Yield Emissions (kg CO2e)')

# --- Step 3: Create a PDF ---
pdf_filename = "annual_report_geodata.pdf"
pdf_pages = PdfPages(pdf_filename)

# Save the plots to PDF
pdf_pages.savefig(fig1)
pdf_pages.savefig(fig2)
pdf_pages.savefig(fig3)

# --- Step 4: Add Insights and Table to PDF ---
def create_pdf_report():
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    c.setFont("Helvetica-Bold", 18)
    c.drawString(72, 750, "Annual Sustainability Report (Geospatial Data)")

    c.setFont("Helvetica", 12)
    c.drawString(72, 730, "Farm Performance and CO2 Emissions Comparison")

    # Add Data Insights
    c.setFont("Helvetica", 10)
    c.drawString(72, 710, f"Data Source: Geospatial Farm Data")
    c.drawString(72, 690, f"- Total emissions decreased from {aggregated_data['Emissions'].iloc[0]:.2f} kg CO2e in {aggregated_data['Year'].iloc[0]} to {aggregated_data['Emissions'].iloc[-1]:.2f} kg CO2e in {aggregated_data['Year'].iloc[-1]}.")
    c.drawString(72, 670, f"- Fuel emissions show a {'decrease' if aggregated_data['FuelEmissions'].iloc[0] > aggregated_data['FuelEmissions'].iloc[-1] else 'slight increase'}.")
    c.drawString(72, 650, f"- Average yield-related emissions improved over the years.")

    # Add Table Data
    c.drawString(72, 630, "Yearly Data Comparison:")
    table_data = [["Year", "Emissions (kg CO2e)", "Fuel Emissions", "Yield Emissions", "Wet Mass", "Dry Matter"]]
    for _, row in aggregated_data.iterrows():
        table_data.append([row['Year'], row['Emissions'], row['FuelEmissions'], row['YieldEmissions'], row['WetMass'], row['DRYMATTER']])

    # Add Table to PDF
    y_position = 600
    for row in table_data:
        for col_num, cell in enumerate(row):
            c.drawString(72 + col_num * 100, y_position, str(cell))
        y_position -= 20

    c.showPage()
    c.save()

# Generate the PDF Report
create_pdf_report()

# Close the PDF pages for charts
pdf_pages.close()

print(f"Annual Report generated and saved as {pdf_filename}")